In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
import quadratic_weighted_kappa
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import feature_generator
import xgboost as xgb
from scipy import optimize
import os.path
from NN import NN
from XgBoost import XGBoostModel
from sklearn.linear_model import Ridge
from CutPoints import CutPointOptimizer

Using Theano backend.


In [34]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    
train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 100, True)
# train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)

In [33]:
# # oldInd = train[train['Ins_Age'] > train['Ins_Age'].median()].index
# ind = train[train['Ins_Age'] <= train['Ins_Age'].median()].index
# train = train.loc[ind]
# # test = test.iloc[ind]
# labels = labels.loc[ind]


AttributeError: 'Series' object has no attribute 'columns'

In [ ]:
def WritePredictionsToFile(model, modelName):
    
    kf = KFold(len(train), 3)
    num = 1
    for train_index, test_index in kf:
        foldFile = 'fold%s.csv' % str(num)
        if os.path.isfile(foldFile):
            predictionsDF = pd.read_csv(foldFile)  
        else:
            predictionsDF = pd.DataFrame()
          
        xTrain = train.iloc[train_index].values
        yTrain = labels.iloc[train_index]      
        model.fit(xTrain, yTrain)
        trainPredictions = model.predict(xTrain)
        
        xValidate = train.iloc[test_index].values
        yValidate = labels.iloc[test_index]
        predictions = model.predict(xValidate)
        predictionsDF[modelName] = predictions
        
        print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
        print quadratic_weighted_kappa.quadratic_weighted_kappa(trainPredictions, yTrain)
        
#         youngTrainPredictions = [trainPredictions[i] for i in range(len(xTrain)) if i in youngInd]
#         oldTrainPredictions = [trainPredictions[i] for i in range(len(trainPredictions)) if i in oldInd]
#         youngYTrain = yTrain.iloc[youngInd] 
                        
        predictionsDF.to_csv(path_or_buf=foldFile, index=False)
   
        predictionsFile = 'testPredictions%s.csv' % str(num)
        if os.path.isfile(predictionsFile):
            testDF = pd.read_csv(predictionsFile)   
        else:
            testDF = pd.DataFrame()
            
        xTest = test.values
        testPredictions = model.predict(xTest)
        testDF[modelName] = testPredictions
        testDF.to_csv(path_or_buf=predictionsFile, index=False)
    
        num += 1

In [ ]:
WritePredictionsToFile(LogisticRegression(), 'LogisticRegression')
# WritePredictionsToFile(XGBoostModel(700, 10, 0.025, 0.65, "reg:linear"), 'XGBoostRegLin')

# WritePredictionsToFile(XGBoostModel(0.3, 1, 0, 700), 'XGBoostLinear')
# WritePredictionsToFile(RandomForestRegressor(n_estimators=10, max_depth=10), 'RandomForest')
# WritePredictionsToFile(BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=20), 'BaggingDescisionTrees_n_estimators=20')
# WritePredictionsToFile(BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0), 'BaggingDescisionTreeClassifiers_n_estimators=20')
# WritePredictionsToFile(BaggingRegressor(base_estimator=LinearRegression(), n_estimators=10), 'BaggingLinearRegression_n_estimators=10')

In [ ]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    
# train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 100)
train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)
WritePredictionsToFile(NN(inputShape = train.shape[1], layers = [100, 50], dropout = [0.5, 0.5], activation='sigmoid', loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 8), 'Keras')

In [ ]:

scorer = make_scorer(quadratic_weighted_kappa.quadratic_weighted_kappa)
# print len(features)
# print len(dummyVariables)

In [ ]:
def qwk_scorer(estimator, X, Y):
    predictions = np.clip(estimator.predict(X), 1, 8)
    return quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.20, random_state=0)
bcf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0)
bcf.fit(X_train, y_train)
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [15, 20, 25], 'max_samples': [0.9, 1.0], 'max_features': [0.9, 1.0]}]
# clf = GridSearchCV(SVR(kernel='rbf', max_iter=1000, epsilon=0.49, tol=0.01, verbose=True), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf = GridSearchCV(BaggingClassifier(base_estimator=DecisionTreeClassifier()), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf.fit(X_train, y_train)
# SVR(kernel='rbf', max_iter=1, , tol=0.01, verbose=True)  
#C=5, g=0.1
# TestQWK: 0.24807920235173586
#C=2, g=0.05, qwk = 0.21
#C=5, g=0.05, qwk = 0.25465662455597715
# do C 

In [ ]:
print clf.best_estimator_

In [ ]:
bcp = bcf.predict(X_test)
print quadratic_weighted_kappa.quadratic_weighted_kappa(bcp, y_test)

In [ ]:
print clf.best_params_
clfPredictions = np.clip(clf.predict(X_test), 1, 8)

# dataPoints = list()

# folds = (2, 5, 10, 20)
# for K in folds:
#     _, testQwk, trainQwk = GetBestModel(GenerateNewCombinedModel, features, K)
#     dataPoints.append((K, testQwk, trainQwk))

In [ ]:
print np.min(clfPredictions)
print np.max(clfPredictions)
print clfPredictions

In [26]:
class CombinedModel:
    
    def __init__(self, modelsFromFile, modelsToCalculate):
        self.modelsToCalculate = modelsToCalculate 
        self.modelsFromFile = modelsFromFile
        self.stackingModel = LinearRegression()
        
    def fit(self, X, Y, fileName):     
        stackingTrainData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))    
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            model.fit(X, Y)
            predictions = model.predict(X)
            stackingTrainData[:,i] = predictions
            df[model] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingTrainData[:,colsToChange] = df[self.modelsFromFile].values
                               
        self.stackingModel.fit(stackingTrainData, Y)
        predictions = self.stackingModel.predict(stackingTrainData)
        
        initialCutPoints = np.array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])
        cpo = CutPointOptimizer(predictions, Y)
        self.cutPoints = optimize.fmin(cpo.qwk, initialCutPoints)

        predictions = np.searchsorted(self.cutPoints, predictions) + 1   

        trainQwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)
        print "Train QWK: %s\n" % trainQwk
                           
    def predict(self, X, fileName):
        stackingData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            predictions = model.predict(X)
            stackingData[:,i] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingData[:,colsToChange] = df[self.modelsFromFile].values
            
        predictions = self.stackingModel.predict(stackingData)
        return np.searchsorted(self.cutPoints, predictions) + 1          

In [30]:
folds = 5
kf = KFold(len(train), folds)
num = 1
# 
combinedModels = list()
qwks = list()
for train_index, test_index in kf:
    
    combinedModel = CombinedModel([], [XGBoostModel(700, 7, 0.025, 0.65, "reg:linear")])
    
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)

    xTrain = train.iloc[train_index].values
    yTrain = labels.iloc[train_index]      
    xValidate = train.iloc[test_index].values
    yValidate = labels.iloc[test_index]

    combinedModel.fit(xTrain, yTrain, trainFile)
    predictions = combinedModel.predict(xValidate, validateFile)
    qwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
    qwks.append(qwk)
    print qwk
    
    combinedModels.append(combinedModel)
    num += 1
    
meanQwk = np.mean(qwks) 
print "Overall Test Qwk: %s" % meanQwk

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.038399
[4]	train-kappa:-0.035474
[5]	train-kappa:-0.027855
[6]	train-kappa:-0.018921
[7]	train-kappa:-0.013438
[8]	train-kappa:-0.010141
[9]	train-kappa:-0.007475
[10]	train-kappa:-0.040664
[11]	train-kappa:-0.050364
[12]	train-kappa:-0.057819
[13]	train-kappa:-0.058374
[14]	train-kappa:-0.058163
[15]	train-kappa:-0.056285
[16]	train-kappa:-0.054175
[17]	train-kappa:-0.051187
[18]	train-kappa:-0.072013
[19]	train-kappa:-0.089906
[20]	train-kappa:-0.100586
[21]	train-kappa:-0.105342
[22]	train-kappa:-0.107509
[23]	train-kappa:-0.110924
[24]	train-kappa:-0.113960
[25]	train-kappa:-0.113880
[26]	train-kappa:-0.112904
[27]	train-kappa:-0.112413
[28]	train-kappa:-0.111114
[29]	train-kappa:-0.138905
[30]	train-kappa:-0.158429
[31]	train-kappa:-0.171208
[32]	train-kappa:-0.179966
[33]	train-kappa:-0.186018
[34]	train-kappa:-0.192255
[35]	train-kappa:-0.196211
[36]	train-kappa:-0.199478
[37]	train-

Optimization terminated successfully.
         Current function value: -0.770235
         Iterations: 168
         Function evaluations: 318
Train QWK: 0.770235155846

Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
0.653107583766

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.038331
[4]	train-kappa:-0.036070
[5]	train-kappa:-0.028432
[6]	train-kappa:-0.020360
[7]	train-kappa:-0.016488
[8]	train-kappa:-0.012753
[9]	train-kappa:-0.009607
[10]	train-kappa:-0.043115
[11]	train-kappa:-0.052804
[12]	train-kappa:-0.058788
[13]	train-kappa:-0.059253
[14]	train-kappa:-0.058813
[15]	train-kappa:-0.056949
[16]	train-kappa:-0.054353
[17]	train-kappa:-0.051287
[18]	train-kappa:-0.062328
[19]	train-kappa:-0.086843
[20]	train-kappa:-0.099821
[21]	train-kappa:-0.105903
[22]	train-kappa:-0.109536
[23]	train-kappa:-0.113186
[24]	train-kappa:-0.114955
[25]	train-kappa:-0.115135
[26]	train-kappa:-0.115040
[27]	train-kappa:-0.115074
[28]	train-kappa:-0.114210
[29]	train-kappa:-0.139133
[30]	train-kappa:-0.156329
[31]	train-kappa:-0.170498
[32]	train-kappa:-0.178854
[33]	train-kappa:-0.187125
[34]	train-kappa:-0.193259
[35]	train-kappa:-0.198863
[36]	train-kappa:-0.203116
[37]	train-


Optimization terminated successfully.
         Current function value: -0.773155
         Iterations: 166
         Function evaluations: 302
Train QWK: 0.773154614661

Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
0.659264681643

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.038704
[4]	train-kappa:-0.036154
[5]	train-kappa:-0.028685
[6]	train-kappa:-0.020360
[7]	train-kappa:-0.016438
[8]	train-kappa:-0.012983
[9]	train-kappa:-0.010012
[10]	train-kappa:-0.044462
[11]	train-kappa:-0.055116
[12]	train-kappa:-0.059748
[13]	train-kappa:-0.059832
[14]	train-kappa:-0.059016
[15]	train-kappa:-0.057228
[16]	train-kappa:-0.054185
[17]	train-kappa:-0.051287
[18]	train-kappa:-0.064766
[19]	train-kappa:-0.088139
[20]	train-kappa:-0.099894
[21]	train-kappa:-0.106553
[22]	train-kappa:-0.110942
[23]	train-kappa:-0.114232
[24]	train-kappa:-0.115156
[25]	train-kappa:-0.115400
[26]	train-kappa:-0.115348
[27]	train-kappa:-0.114986
[28]	train-kappa:-0.114598
[29]	train-kappa:-0.136342
[30]	train-kappa:-0.156507
[31]	train-kappa:-0.173303
[32]	train-kappa:-0.182355
[33]	train-kappa:-0.189094
[34]	train-kappa:-0.195173
[35]	train-kappa:-0.201543
[36]	train-kappa:-0.205717
[37]	train-


Optimization terminated successfully.
         Current function value: -0.769317
         Iterations: 146
         Function evaluations: 288
Train QWK: 0.769316599574

Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
0.654757103666

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.038019
[4]	train-kappa:-0.035789
[5]	train-kappa:-0.028268
[6]	train-kappa:-0.020504
[7]	train-kappa:-0.016726
[8]	train-kappa:-0.013216
[9]	train-kappa:-0.010014
[10]	train-kappa:-0.044916
[11]	train-kappa:-0.053884
[12]	train-kappa:-0.059267
[13]	train-kappa:-0.059167
[14]	train-kappa:-0.058530
[15]	train-kappa:-0.056356
[16]	train-kappa:-0.053442
[17]	train-kappa:-0.050410
[18]	train-kappa:-0.053577
[19]	train-kappa:-0.088920
[20]	train-kappa:-0.101138
[21]	train-kappa:-0.106372
[22]	train-kappa:-0.110331
[23]	train-kappa:-0.114167
[24]	train-kappa:-0.115145
[25]	train-kappa:-0.115368
[26]	train-kappa:-0.115365
[27]	train-kappa:-0.114441
[28]	train-kappa:-0.113819
[29]	train-kappa:-0.135752
[30]	train-kappa:-0.157278
[31]	train-kappa:-0.172161
[32]	train-kappa:-0.181072
[33]	train-kappa:-0.187563
[34]	train-kappa:-0.193838
[35]	train-kappa:-0.199622
[36]	train-kappa:-0.203428
[37]	train-


Optimization terminated successfully.
         Current function value: -0.772454
         Iterations: 179
         Function evaluations: 346
Train QWK: 0.772453647771

Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
0.65810566057

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.037928
[4]	train-kappa:-0.035834
[5]	train-kappa:-0.027705
[6]	train-kappa:-0.020410
[7]	train-kappa:-0.016364
[8]	train-kappa:-0.012669
[9]	train-kappa:-0.009722
[10]	train-kappa:-0.043844
[11]	train-kappa:-0.053895
[12]	train-kappa:-0.059363
[13]	train-kappa:-0.059530
[14]	train-kappa:-0.058721
[15]	train-kappa:-0.056731
[16]	train-kappa:-0.053914
[17]	train-kappa:-0.050684
[18]	train-kappa:-0.054554
[19]	train-kappa:-0.087102
[20]	train-kappa:-0.101536
[21]	train-kappa:-0.104820
[22]	train-kappa:-0.108178
[23]	train-kappa:-0.112972
[24]	train-kappa:-0.114738
[25]	train-kappa:-0.114874
[26]	train-kappa:-0.115235
[27]	train-kappa:-0.115007
[28]	train-kappa:-0.113710
[29]	train-kappa:-0.133740
[30]	train-kappa:-0.155525
[31]	train-kappa:-0.169944
[32]	train-kappa:-0.180194
[33]	train-kappa:-0.186473
[34]	train-kappa:-0.192899
[35]	train-kappa:-0.197883
[36]	train-kappa:-0.201212
[37]	train-


Optimization terminated successfully.
         Current function value: -0.772834
         Iterations: 192
         Function evaluations: 347
Train QWK: 0.772834095754

Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
0.667443170935
Overall Test Qwk: 0.658535640116


In [35]:
testPredictions = np.zeros(len(test))
for i in range(1, len(combinedModels) + 1):
    trainFile = 'combinedTrainPredictions%s.csv' % str(i)
    testPredictions += combinedModels[i-1].predict(test, trainFile)
    
testPredictions /= folds
predDf = pd.DataFrame()
predDf['Id'] = dfTest['Id']
predDf['Response'] = np.round(testPredictions).astype(int)
print predDf['Response'].values
predDf.to_csv(path_or_buf='Take2.csv', columns=['Id', 'Response'], index=False, header=['Id', 'Response'])

Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
Index([u'LogisticRegression', u'XgBoostRegLin', u'XgBoostRegLin_Depth8',
       u'XgBoostRegLin_Depth7'],
      dtype='object')
[3 6 7 ..., 2 1 2]


In [11]:
kf = KFold(len(train), 5)
num = 1
qwks = list()

modelToCalcNames = ['XgBoostRegLin_Depth7']

for train_index, test_index in kf:
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)
    
    trainDF = pd.read_csv(trainFile) if os.path.isfile(trainFile) else pd.DataFrame()  
    validateDF = pd.read_csv(validateFile) if os.path.isfile(validateFile) else pd.DataFrame()  

    xTrain = train.iloc[train_index].values
    yTrain = labels.iloc[train_index]      
    xValidate = train.iloc[test_index].values
    yValidate = labels.iloc[test_index]
    
    for i in range (len(combinedModel.modelsToCalculate)):
        model = combinedModel.modelsToCalculate[i] 
        model.fit(xTrain, yTrain)
        trainPredictions = model.predict(xTrain)
        validatePredictions = model.predict(xValidate)
        name = modelToCalcNames[i]  
        trainDF[name] = trainPredictions
        validateDF[name] = validatePredictions
        
    trainDF.to_csv(path_or_buf=trainFile, index=False)
    validateDF.to_csv(path_or_buf=validateFile, index=False)
            
    num += 1

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.038399
[4]	train-kappa:-0.035474
[5]	train-kappa:-0.027855
[6]	train-kappa:-0.018921
[7]	train-kappa:-0.013438
[8]	train-kappa:-0.010141
[9]	train-kappa:-0.007475
[10]	train-kappa:-0.040664
[11]	train-kappa:-0.050364
[12]	train-kappa:-0.057819
[13]	train-kappa:-0.058374
[14]	train-kappa:-0.058163
[15]	train-kappa:-0.056285
[16]	train-kappa:-0.054175
[17]	train-kappa:-0.051187
[18]	train-kappa:-0.072013
[19]	train-kappa:-0.089906
[20]	train-kappa:-0.100586
[21]	train-kappa:-0.105342
[22]	train-kappa:-0.107509
[23]	train-kappa:-0.110924
[24]	train-kappa:-0.113960
[25]	train-kappa:-0.113880
[26]	train-kappa:-0.112904
[27]	train-kappa:-0.112413
[28]	train-kappa:-0.111114
[29]	train-kappa:-0.138905
[30]	train-kappa:-0.158429
[31]	train-kappa:-0.171208
[32]	train-kappa:-0.179966
[33]	train-kappa:-0.186018
[34]	train-kappa:-0.192255
[35]	train-kappa:-0.196211
[36]	train-kappa:-0.199478
[37]	train-

In [ ]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

keywords = ['Medical_Keyword_' + str(i) for i in range(1, 49)]
# print dfTrain[keywords].sum(axis=1)

# for keyword in keywords:
#     print pearsonr(dfTrain[keyword], dfTrain.Response)
#     break

# uniqueValues = dfTest['InsuredInfo_7'].unique()
# for i in range(len(uniqueValues)):
#     arr = dfTrain['InsuredInfo_7'].apply(lambda x: x == uniqueValues[i])
    

for column in ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5']:
   
    if dfTrain[column].isnull().sum():
        print column
        print pearsonr(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
        print dfTrain[column].median()
        print len(dfTrain[dfTrain.Response == 8])
        print len(dfTrain)
#         print dfTrain[column]
#         plt.plot(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
        break
# plt.show()
# print dfTest['InsuredInfo_4'].unique()
# print dfTrain['InsuredInfo_6'].apply(lambda x: x == 1)
print dfTest['Medical_History_32'].isnull().sum()
            
print dfTrain['Medical_History_32'].median()
print dfTrain[dfTrain['Family_Hist_2'].notnull()]['Family_Hist_2'].max()
# print dfTrain['Medical_History_32'].null().sum()
# print dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'].median()
# print len(pd.concat([dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32']]))
# print pearsonr(dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Response'])

In [ ]:
neighborModel = GetBestModel(lambda: NearestNeighbors(), features)

In [ ]:
nearestNeighbors = neighborModel.kneighbors(dfTrain[features].values)

In [ ]:
predictions = np.zeros(len(dfTrain))
for i in range(len(dfTrain)):
#     predictions[i] = dfTrain.iloc[nearestNeighbors[1][i]].Response.mean()
    responses = dfTrain.iloc[nearestNeighbors[1][i]].Response
    weights = nearestNeighbors[0][i][4] - nearestNeighbors[0][i]
    predictions[i] = np.sum(responses * weights) / np.sum(weights)

In [ ]:
predictions = np.rint(predictions)
print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, dfTrain.Response)
print np.min(predictions)

In [ ]:
import scipy.stats as stats
lowerBound = np.mean(dfTrain['BMI']) - np.std(dfTrain['BMI'])
upperBound = np.mean(dfTrain['BMI']) + np.std(dfTrain['BMI'])
mean = np.mean(dfTrain['BMI'])
stats.normaltest(dfTrain['BMI'])

bmiScore = dfTrain['BMI'].apply(lambda x: max(0, lowerBound - x) if x < mean else max(0, x - upperBound))

model = LinearRegression(normalize=True)
features = ['Ins_Age', 'Wt', 'BMI']
X = np.ndarray((59381, 9))
X[:,0] = dfTrain['Ins_Age'].values
X[:,1] = dfTrain['BMI'].values
X[:,2] = dfTrain['Wt'].values
X[:,3] = dfTrain['Ins_Age'] * dfTrain['Wt']
X[:,4] = dfTrain['Ins_Age'] * dfTrain['BMI']
X[:,5] = dfTrain['Ins_Age'] * dfTrain['Ins_Age']
X[:,6] = dfTrain['Wt'] * dfTrain['Wt']
X[:,7] = dfTrain['Wt'] * dfTrain['BMI']
X[:,8] = dfTrain['Wt'] * dfTrain['Ins_Age']

model.fit(X, dfTrain['Response'])

In [ ]:
class CutPointOptimizer:
    
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual

    def qwk(self, cutPoints):
        transformedPredictions = np.searchsorted(cutPoints, self.predicted) + 1            
        return -1 * quadratic_weighted_kappa.quadratic_weighted_kappa(transformedPredictions, self.actual)

initialCutPoints = np.array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])

In [ ]:

# Product_Info_2
# Product_Info_3
# Employment_Info_2
# InsuredInfo_3
# Medical_History_2
# Medical_History_10

var = 'Product_Info_3'
for val in dfTrain[var].unique():
    hits = dfTrain[dfTrain[var] == val]
    print '%s, %s, %s' % (val, len(hits), np.mean(hits.Response))

In [ ]:
K = 3
kf = KFold(len(train), 3)
